In [1]:
import ctypes

import numpy
from numpy import ndarray, int32
from numpy.ctypeslib import as_ctypes, ndpointer

In [ ]:
def infer_simple_ctype(var):
    if isinstance(var, int):
        return ctypes.c_int

    elif isinstance(var, float):
        return ctypes.c_double

    elif isinstance(var, bool):
        return ctypes.c_bool

    elif isinstance(var, str):
        return ctypes.c_char_p

    else:
        raise NotImplementedError("Cannot infer ctype of type(var)={:}, var={:}".format(type(var), var))

In [ ]:
def preprocess_types(input_tuple, output_tuple):
    if len(output_tuple) > 1:
        raise NotImplementedError("C does not support multiple return values but len(output_tuple)={:d}"
                                  .format(len(output_tuple)))

    input_list = []
    arg_ctypes = []

    for var in input_tuple:
        if isinstance(var, str):
            arg_ctypes.append(ctypes.c_char_p)

            # C wants bytes, not strings.
            c_str = bytes(var, "utf-8")
            input_list.append(ctypes.c_char_p(c_str))

        elif isinstance(var, list):
            if isinstance(var[0], (list, tuple)):
                raise NotImplementedError(f"Cannot infer ctype of a list containing lists or tuples: var={var}")

            arr_ctype = infer_simple_ctype(var[0]) * len(var)
            arg_ctypes.append(arr_ctype)

            arr = arr_ctype(*var)
            input_list.append(arr)

            # For a Python list, we add an extra argument for the size of the C array.
            arg_ctypes.append(ctypes.c_int)
            input_list.append(len(var))

        elif isinstance(var, ndarray):
            arr_ctype = ndpointer(ctypes.c_double, flags="C_CONTIGUOUS")
            arg_ctypes.append(arr_ctype)
            input_list.append(var)

            # For a numpy ndarray, we add extra arguments for each dimension size of the input C array.
            for s in range(len(var.shape)):
                arg_ctypes.append(ctypes.c_int)
                input_list.append(s)

        else:
            arg_ctypes.append(infer_simple_ctype(var))
            input_list.append(var)

    rvar = output_tuple[0]  # Return variable

    if isinstance(rvar, list):
        # If the C function needs to return an array, Python must allocate memory for the array and pass it to the
        # C function. So we add an extra argument for a pointer to the pre-allocated C array and set the return type
        # to void.
        if isinstance(var[0], (list, tuple)):
            raise NotImplementedError(f"Cannot infer ctype of a list containing lists or tuples: var={var}")

        arr_ctype = infer_simple_ctype(rvar[0]) * len(rvar)
        arg_ctypes.append(arr_ctype)

        arr = arr_ctype()
        input_list.append(arr)

        res_ctype = ctypes.c_void_p
    else:
        res_ctype = infer_simple_ctype(rvar)

    return arg_ctypes, res_ctype, input_list

In [ ]:
it = [1, 2, 3, 4, 5], 10
preprocess_types(it, (100,))

In [ ]:
it = numpy.array([1, 2, 3, 4, 5]), 10
preprocess_types(it, (10.0,))

In [ ]:
var = numpy.random.uniform(-1, 1, size=5)
ndpointer(dtype=var.dtype, flags="C_CONTIGUOUS")

In [ ]:
var = numpy.array([1, 7, 11])
ndpointer(dtype=var.dtype, flags="C_CONTIGUOUS")

In [ ]:
numpy.random.rand(3).shape

In [ ]:
preprocess_types(("ABC",), (1,))

In [ ]:
cs = ctypes.c_char_p(b"ABC")

In [ ]:
isinstance(cs, ctypes.c_char_p)

In [ ]:
numpy.zeros(numpy.random.rand(3).shape, dtype=numpy.int32)

In [2]:
import ctypes

import numpy
from numpy import ndarray, int32
from numpy.ctypeslib import as_ctypes, ndpointer

_lib = ctypes.cdll.LoadLibrary("/home/alir/tmp/libchaos.so")

f51 = numpy.zeros(51, dtype=numpy.float64)
_lib.logistic_map.argtypes = [ctypes.c_double, ndpointer(dtype=f51.dtype, flags="C_CONTIGUOUS")]
_lib.logistic_map.restype = ctypes.c_void_p

_lib.logistic_map(1.0, f51)

In [21]:
f51 = numpy.zeros(51, dtype=numpy.float64)
_lib.logistic_map.argtypes = [ctypes.c_float, ndpointer(dtype=f51.dtype, flags="C_CONTIGUOUS")]
_lib.logistic_map.restype = ctypes.c_void_p

In [22]:
f51

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [23]:
_lib.logistic_map(1.0, f51)

94144326783064

In [24]:
f51

array([6.91903211e-310, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000])

In [26]:
numpy.allclose([1, 3, 4], numpy.array([1, 3, 4.5]))

False

In [ ]:
import ctypes

import numpy
from numpy import ndarray
from numpy.ctypeslib import as_ctypes, ndpointer

_lib = ctypes.cdll.LoadLibrary("/home/alir/tmp/libtemp.so")

T = numpy.random.uniform(1, 10, size=17)

_lib.temperature_statistics.argtypes = [ndpointer(dtype=T.dtype, flags="C_CONTIGUOUS"), ctypes.c_int, ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double)]
_lib.temperature_statistics.restype = ctypes.c_void_p

m, s = ctypes.c_double(-1.5), ctypes.c_double(-2.5)
_lib.temperature_statistics(T, len(T), m, s)

In [4]:
m

c_double(0.0)

In [ ]:
import ctypes

import numpy
from numpy import ndarray
from numpy.ctypeslib import as_ctypes, ndpointer

_lib = ctypes.cdll.LoadLibrary("/home/alir/tmp/liblife.so")

B = numpy.random.randint(0, 2, size=(5, 7))
N, M = B.shape
FB = numpy.zeros(B.shape, dtype=B.dtype)

B_ptr_t = ndpointer(dtype=B.dtype, ndim=len(B.shape), shape=B.shape, flags="C_CONTIGUOUS")
FB_ptr_t = ndpointer(dtype=FB.dtype, ndim=len(FB.shape), shape=FB.shape, flags="C_CONTIGUOUS")

_lib.game_of_life.argtypes = [B_ptr_t, ctypes.c_int, ctypes.c_int, ctypes.c_int, FB_ptr_t]
_lib.game_of_life.restype = ctypes.c_void_p
_lib.game_of_life(B, N, M, 30, FB)

In [7]:
import ctypes

import numpy as np
from numpy import ndarray
from numpy.ctypeslib import as_ctypes, ndpointer

_lib = ctypes.cdll.LoadLibrary("/home/alir/tmp/liblife.so")

B = np.random.randint(0, 2, size=(5, 7))
N, M = B.shape
FB = np.zeros(B.shape, dtype=B.dtype)

B_pp = (B.ctypes.data + np.arange(B.shape[0]) * B.strides[0]).astype(np.uintp)
B_ptr_t = np.ctypeslib.ndpointer(dtype=np.uintp)

FB_pp = (FB.ctypes.data + np.arange(B.shape[0]) * FB.strides[0]).astype(np.uintp)
FB_ptr_t = np.ctypeslib.ndpointer(dtype=np.uintp)

_lib.game_of_life.argtypes = [B_ptr_t, ctypes.c_int, ctypes.c_int, ctypes.c_int, FB_ptr_t]
_lib.game_of_life.restype = ctypes.c_void_p
_lib.game_of_life(B_pp, N, M, 30, FB_pp)

numpy.ctypeslib.ndpointer_<i8_2d_5x7_C_CONTIGUOUS

2